**Задание 2.**

Мы всё ещё не смотрим в контент, хотя его тут очень много. Попробуйте по этим данным сделать рекомендацию на основе контента.

Идея тут может быть такой: смотрим на описание фильмов тэгами, а когда пользователь ставит оценку за фильм, мы её засчитываем как оценку за тэги.

In [ ]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install PyDrive

In [ ]:
from google.colab import auth
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
from pydrive.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1dGbV43VYV8yzHeZHxvIsVIGYzyzukjWd"}) 
downloaded.GetContentFile('ratings.csv')
rating=pd.read_csv('ratings.csv')

In [ ]:
downloaded = drive.CreateFile({'id':"1W0L7Vr02vT5PSGWGMk4BLtSrg-aFaP75"}) 
downloaded.GetContentFile('movies.csv')
movies=pd.read_csv('movies.csv')

In [ ]:
downloaded = drive.CreateFile({'id':"1Udg_1EuRU299LdX09Vkz9x5-hKMbxLl5"}) 
downloaded.GetContentFile('genome-tags.csv')
tags=pd.read_csv('genome-tags.csv')

In [ ]:
downloaded = drive.CreateFile({'id':"108PsHwMEnzWprOwTxh-zCseF0unGxuhl"}) 
downloaded.GetContentFile('genome-scores.csv')
scores=pd.read_csv('genome-scores.csv')

Составим сводную таблицу фильмов и их тегов.

In [ ]:
movies_tags = pd.merge(scores, tags,on = 'tagId', how = 'left')
movies_with_tags = pd.merge(movies, movies_tags)
movies_with_tags = movies_with_tags.drop(['genres', 'tagId'], axis = 1)

In [ ]:
movies_with_tags.head()

,movieId,title,relevance,tag
0,1,Toy Story (1995),0.02875,007
1,1,Toy Story (1995),0.02375,007 (series)
2,1,Toy Story (1995),0.06250,18th century
3,1,Toy Story (1995),0.07575,1920s
4,1,Toy Story (1995),0.14075,1930s


Далее на примере одного пользователя те оценки, которые он ставил за фильмы, засчитаем как оценку за теги.

Чтобы получить взвешенную оценку, оценку пользователя будем умножать на коэффициент релеватности тега.


In [ ]:
data = pd.merge(movies, rating)
data = data.drop(columns = ['genres', 'timestamp'], axis = 1)

In [ ]:
all_users = rating['userId'].value_counts()
user = all_users.sample(1).index
user

Int64Index([92104], dtype='int64')

In [ ]:
user_movies = data[data.userId == user[0]]

user_scores = pd.merge(movies_with_tags, user_movies, how = 'inner', on = 'movieId')
user_scores = user_scores.drop(columns='title_y', axis = 1)

user_scores['score'] = user_scores['rating'] * user_scores['relevance']
user_scores.head()

,movieId,title_x,relevance,tag,userId,rating,score
0,158,Casper (1995),0.03850,007,92104,3.0,0.11550
1,158,Casper (1995),0.04450,007 (series),92104,3.0,0.13350
2,158,Casper (1995),0.06575,18th century,92104,3.0,0.19725
3,158,Casper (1995),0.04025,1920s,92104,3.0,0.12075
4,158,Casper (1995),0.05325,1930s,92104,3.0,0.15975


Оставляем только те теги, где score больше 3.5.

In [ ]:
user_scores = user_scores[user_scores.score >= 3.5]

In [ ]:
user_scores

,movieId,title_x,relevance,tag,userId,rating,score
2316,455,Free Willy (1993),0.92800,animal movie,92104,4.0,3.712
2317,455,Free Willy (1993),0.90900,animals,92104,4.0,3.636
2458,455,Free Willy (1993),0.92200,childhood,92104,4.0,3.688
2459,455,Free Willy (1993),0.95025,children,92104,4.0,3.801
2629,455,Free Willy (1993),0.97100,family,92104,4.0,3.884
...,...,...,...,...,...,...,...
56904,48322,Jackass Number Two (2006),0.94925,hilarious,92104,4.0,3.797
56932,48322,Jackass Number Two (2006),0.87625,idiotic,92104,4.0,3.505
57024,48322,Jackass Number Two (2006),0.98875,male nudity,92104,4.0,3.955
57077,48322,Jackass Number Two (2006),0.90225,mtv,92104,4.0,3.609


Теги ранжируем по score и оставляем топ-k уникальных. Возьмем 30.


In [ ]:
user_scores = user_scores.sort_values(by = 'score', ascending= False)
user_scores

,movieId,title_x,relevance,tag,userId,rating,score
44126,40819,Walk the Line (2005),0.99125,biopic,92104,5.0,4.95625
44124,40819,Walk the Line (2005),0.98225,biographical,92104,5.0,4.91125
41868,37741,Capote (2005),0.98150,biographical,92104,5.0,4.90750
44738,40819,Walk the Line (2005),0.97500,oscar (best actress),92104,5.0,4.87500
44678,40819,Walk the Line (2005),0.96925,musicians,92104,5.0,4.84625
...,...,...,...,...,...,...,...
50701,44665,Lucky Number Slevin (2006),0.87675,vengeance,92104,4.0,3.50700
45861,41285,Match Point (2005),0.77900,original,92104,4.5,3.50550
7348,946,To Be or Not to Be (1942),0.87625,jews,92104,4.0,3.50500
56932,48322,Jackass Number Two (2006),0.87625,idiotic,92104,4.0,3.50500


In [ ]:
user_tags = user_scores['tag'].unique()[:30]

In [ ]:
user_tags

array(['biopic', 'biographical', 'oscar (best actress)', 'musicians',
       'oscar (best actor)', 'based on a book', 'character study',
       'great acting', 'drug addiction', 'music', 'biography',
       "writer's life", 'oscar (best directing)', 'murder', 'writers',
       'literature', 'drug abuse', 'based on book', 'dynamic cgi action',
       'adapted from:book', 'truman capote', 'imdb top 250', 'sports',
       'football', 'disability', 'spelling bee', 'death penalty',
       'eccentricity', 'great music', 'big budget'], dtype=object)

Далее из непросмотренных пользователем фильмов выбираем те, в которых присутствует хотя бы один из этих тегов и его релевантность не ниже 0.65.



In [ ]:
movies_with_tags = movies_with_tags[movies_with_tags.relevance >= 0.65]

In [ ]:
recommendations = {}

for i in movies_with_tags:
   for index, row in movies_with_tags.iterrows():
     if row['movieId'] not in user_movies and row['tag'] in user_tags:
         recommendations[row['title']] = row['tag']

In [ ]:
len(recommendations)

6428

Мы получили довольно много рекомендаций, даже если учесть, что часть фильмов может дублироваться под разными тегами. Уже на этом этапе мы можем рекомендовать пользователю фильмы по конкретным тегам.

In [ ]:
def get_film(d, value):
  film_by_tag = []
  for k, v in d.items():
    if v == value:
      film_by_tag.append(k)
  return film_by_tag

In [ ]:
user_tags[0]

'biopic'

In [ ]:
get_film(recommendations, user_tags[0])

['Wild Bill (1995)',
 'Wyatt Earp (1994)',
 'Shadowlands (1993)',
 'Michael Collins (1996)',
 'Surviving Picasso (1996)',
 'Private Parts (1997)',
 'Artemisia (1997)',
 'Kundun (1997)',
 'Why Do Fools Fall In Love? (1998)',
 'General, The (1998)',
 'Mommie Dearest (1981)',
 'Stand and Deliver (1988)',
 'Steal This Movie! (2000)',
 'Rembrandt (1936)',
 'Tucker: The Man and His Dream (1988)',
 'MacArthur (1977)',
 'Piñero (2001)',
 'Dragon: The Bruce Lee Story (1993)',
 'Painted Fire (Chihwaseon) (2002)',
 'Desert Fox, The (1951)',
 'Luther (2003)',
 'Brother Sun, Sister Moon (Fratello sole, sorella luna) (1972)',
 'Lola Montès (1955)',
 'Moulin Rouge (1952)',
 'Krays, The (1990)',
 'Trinity and Beyond (1995)',
 'Libertine, The (2004)',
 'Notorious Bettie Page, The (2005)',
 'Fur: An Imaginary Portrait of Diane Arbus (2006)',
 'Amazing Grace (2006)',
 "Goya's Ghosts (2006)",
 'Talk to Me (2007)',
 'Mongol (2007)',
 'W. (2008)',
 'Flash of Genius (2008)',
 'Che: Part One (2008)',
 'Che: P

Все еще много рекомендаций, и непонятно какая из них наиболее предпочтительна. 

Если считать общий рейтинг фильма, например, на TMDB, тоже его внутренней характеристикой, то мы можем выдавать только фильмы с рейтингом не ниже 6.5 (по 10балльной шкале).

Воспользуемся датасетом movies_metadata по movieLens. https://www.kaggle.com/rounakbanik/the-movies-dataset, где есть показатель vote_average.

In [ ]:
downloaded = drive.CreateFile({'id':'15a_S2GFSdZHCZejSZVmqa5xxvVYOxUbO'}) 
downloaded.GetContentFile('movies_metadata.csv')
movies_metadata = pd.read_csv('movies_metadata.csv')

In [ ]:
movies_metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
tmdb_rating = movies_metadata[['id', 'vote_average']]
tmdb_rating = tmdb_rating.rename(columns = {'id':'movieId'})

Оставляем только фильмы со средним рейтингом выше 6.5. И согласно нему выдаем рекомендации.


In [ ]:
top_tmdb_rating = tmdb_rating[tmdb_rating.vote_average >= 6.5]
top_tmdb_rating = top_tmdb_rating.apply(lambda col:pd.to_numeric(col, errors='coerce'))

In [ ]:
movies_with_tags = pd.merge(movies_with_tags, top_tmdb_rating)

In [ ]:
movies_with_tags

,movieId,title,relevance,tag,vote_average
0,2,Jumanji (1995),0.97600,adventure,7.1
1,2,Jumanji (1995),0.86725,animals,7.1
2,2,Jumanji (1995),0.81750,based on a book,7.1
3,2,Jumanji (1995),0.77525,big budget,7.1
4,2,Jumanji (1995),0.86850,childhood,7.1
...,...,...,...,...,...
42471,198185,Twin Peaks (1989),0.95925,weird,6.6
42472,201749,Murder Mystery (2019),0.68950,comedy,6.9
42473,201749,Murder Mystery (2019),0.69075,family,6.9
42474,201749,Murder Mystery (2019),0.77400,goofy,6.9


In [ ]:
final_recommendations = {}

for i in movies_with_tags:
   for index, row in movies_with_tags.iterrows():
           if row['movieId'] not in user_movies and row['tag'] in user_tags:
               final_recommendations[row['title']] = row['tag']

In [ ]:
len(final_recommendations)

912

Уже лучше, посмотрим как теперь выглядит наш список по тому же тегу.

In [ ]:
get_film(final_recommendations, user_tags[0])

['Kundun (1997)',
 'Mommie Dearest (1981)',
 'MacArthur (1977)',
 'Dragon: The Bruce Lee Story (1993)',
 "Goya's Ghosts (2006)",
 'W. (2008)',
 'Che: Part One (2008)',
 'Séraphine (2008)',
 'Little Ashes (2008)',
 'Yves Saint Laurent (2014)']

Content-based рекомендации можно развивать и далее, учитывая жанры и другие метаданные (например, не average vote, а popularity).